为了运行以下笔记本，如果您还没有这样做，您需要部署一个使用 `text-embedding-ada-002` 作为基础模型的模型，并在 .env 文件中将部署名称设置为 `AZURE_OPENAI_EMBEDDINGS_ENDPOINT`。


In [ ]:
import os
import pandas as pd
import numpy as np
from openai import AzureOpenAI
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
  api_key=os.environ['AZURE_OPENAI_API_KEY'],  # this is also the default, it can be omitted
  api_version = "2023-05-15"
  )

model = os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']

SIMILARITIES_RESULTS_THRESHOLD = 0.75
DATASET_NAME = "../embedding_index_3m.json"

接下来，我们将把嵌入索引加载到 Pandas 数据框中。嵌入索引存储在一个名为 `embedding_index_3m.json` 的 JSON 文件中。嵌入索引包含截至 2023 年 10 月底的每个 YouTube 转录文本的嵌入。


In [ ]:
def load_dataset(source: str) -> pd.core.frame.DataFrame:
    # Load the video session index
    pd_vectors = pd.read_json(source)
    return pd_vectors.drop(columns=["text"], errors="ignore").fillna("")

接下来，我们将创建一个名为 `get_videos` 的函数，该函数将在嵌入索引中搜索查询。该函数将返回与查询最相似的前5个视频。函数的工作流程如下：

1. 首先，创建嵌入索引的副本。
2. 接着，使用 OpenAI 嵌入 API 计算查询的嵌入。
3. 然后，在嵌入索引中创建一个名为 `similarity` 的新列。`similarity` 列包含查询嵌入与每个视频片段嵌入之间的余弦相似度。
4. 接下来，根据 `similarity` 列过滤嵌入索引。嵌入索引仅保留余弦相似度大于或等于0.75的视频。
5. 最后，根据 `similarity` 列对嵌入索引进行排序，并返回前5个视频。


In [ ]:
def cosine_similarity(a, b):
    if len(a) > len(b):
        b = np.pad(b, (0, len(a) - len(b)), 'constant')
    elif len(b) > len(a):
        a = np.pad(a, (0, len(b) - len(a)), 'constant')
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def get_videos(
    query: str, dataset: pd.core.frame.DataFrame, rows: int
) -> pd.core.frame.DataFrame:
    # create a copy of the dataset
    video_vectors = dataset.copy()

    # get the embeddings for the query    
    query_embeddings = client.embeddings.create(input=query, model=model).data[0].embedding

    # create a new column with the calculated similarity for each row
    video_vectors["similarity"] = video_vectors["ada_v2"].apply(
        lambda x: cosine_similarity(np.array(query_embeddings), np.array(x))
    )

    # filter the videos by similarity
    mask = video_vectors["similarity"] >= SIMILARITIES_RESULTS_THRESHOLD
    video_vectors = video_vectors[mask].copy()

    # sort the videos by similarity
    video_vectors = video_vectors.sort_values(by="similarity", ascending=False).head(
        rows
    )

    # return the top rows
    return video_vectors.head(rows)

这个函数非常简单，它只是打印出搜索查询的结果。


In [ ]:
def display_results(videos: pd.core.frame.DataFrame, query: str):
    def _gen_yt_url(video_id: str, seconds: int) -> str:
        """convert time in format 00:00:00 to seconds"""
        return f"https://youtu.be/{video_id}?t={seconds}"

    print(f"\nVideos similar to '{query}':")
    for _, row in videos.iterrows():
        youtube_url = _gen_yt_url(row["videoId"], row["seconds"])
        print(f" - {row['title']}")
        print(f"   Summary: {' '.join(row['summary'].split()[:15])}...")
        print(f"   YouTube: {youtube_url}")
        print(f"   Similarity: {row['similarity']}")
        print(f"   Speakers: {row['speaker']}")

1. 首先，将嵌入索引加载到 Pandas Dataframe 中。  
2. 接下来，提示用户输入查询。  
3. 然后调用 `get_videos` 函数在嵌入索引中搜索查询。  
4. 最后，调用 `display_results` 函数向用户显示结果。  
5. 然后提示用户输入另一个查询。此过程将持续，直到用户输入 `exit`。  

![](../../../../translated_images/notebook-search.1e320b9c7fcbb0bc1436d98ea6ee73b4b54ca47990a1c952b340a2cadf8ac1ca.zh.png)

系统将提示您输入查询。输入查询并按回车。应用程序将返回与查询相关的视频列表。应用程序还将返回视频中答案所在位置的链接。

以下是一些可尝试的查询：

- 什么是 Azure 机器学习？  
- 卷积神经网络是如何工作的？  
- 什么是神经网络？  
- 我可以将 Jupyter 笔记本与 Azure 机器学习一起使用吗？  
- 什么是 ONNX？


In [ ]:
pd_vectors = load_dataset(DATASET_NAME)

# get user query from input
while True:
    query = input("Enter a query: ")
    if query == "exit":
        break
    videos = get_videos(query, pd_vectors, 5)
    display_results(videos, query)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**免责声明**：  
本文件由 AI 翻译服务 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻译而成。尽管我们力求准确，但请注意自动翻译可能存在错误或不准确之处。原始文件的母语版本应被视为权威来源。对于重要信息，建议使用专业人工翻译。因使用本翻译而产生的任何误解或误释，我们概不负责。
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
